In [1]:
import tensorflow_datasets as tfds

In [2]:
imdb, info = tfds.load("imdb_reviews", with_info=True, as_supervised= True)

In [3]:
import numpy as np
import tensorflow as tf

In [4]:
train_data, test_data = imdb["train"], imdb["test"]

In [5]:
training_sentences = []
training_labels = []

In [6]:
testing_sentences =[]
testing_labels = []

In [7]:
for s, l in train_data:
        training_sentences.append(s.numpy().decode('utf8'))
        training_labels.append(l.numpy())
    
for s, l in test_data:
          testing_sentences.append(s.numpy().decode('utf8'))
          testing_labels.append(l.numpy())
    

In [8]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

In [9]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type = 'post'
oov_tok = "<OOV>" 

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [11]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok )
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences= tokenizer.texts_to_sequences(training_sentences)

padded = pad_sequences(sequences, maxlen= max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [13]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [14]:
num_epochs = 10
model.fit( padded, training_labels_final, epochs=num_epochs, validation_data=(test_padded, testing_labels_final ))

Epoch 1/10
782/782 [==============================] - 4s 4ms/step - loss: 0.6031 - accuracy: 0.6383 - val_loss: 0.3467 - val_accuracy: 0.8437
Epoch 2/10
782/782 [==============================] - 2s 3ms/step - loss: 0.2499 - accuracy: 0.9029 - val_loss: 0.3745 - val_accuracy: 0.8380
Epoch 3/10
782/782 [==============================] - 2s 3ms/step - loss: 0.1111 - accuracy: 0.9704 - val_loss: 0.4387 - val_accuracy: 0.8290
Epoch 4/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0302 - accuracy: 0.9967 - val_loss: 0.5177 - val_accuracy: 0.8278
Epoch 5/10
782/782 [==============================] - 2s 3ms/step - loss: 0.0069 - accuracy: 0.9997 - val_loss: 0.6021 - val_accuracy: 0.8253
Epoch 6/10
782/782 [==============================] - 3s 4ms/step - loss: 0.0024 - accuracy: 1.0000 - val_loss: 0.6413 - val_accuracy: 0.8296
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 9.9995e-04 - accuracy: 1.0000 - val_loss: 0.6823 - val_accuracy: 0.8307
Ep

In [23]:
reverse_word_index = dict([(value, key ) for (key,value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(decode_review(padded[3]))
print()
print(training_sentences[3])

? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? this is the kind of film for a snowy sunday afternoon when the rest of the world can go ahead with its own business as you <OOV> into a big arm chair and <OOV> for a couple of hours wonderful performances from cher and nicolas cage as always gently row the plot along there are no <OOV> to cross no dangerous waters just a warm and witty <OOV> through new york life at its best a family film in every sense and one that deserves the praise it received

This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful performances from Cher and Nicolas Cage (as always) gently row the plot along. There are no rapids to cross, no dangerous waters, just a warm and witty paddle through New York life at its best. A family film in every sense and one that deserves the praise it received.


In [20]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 16)


In [21]:
import io

out_v = io.open('vec.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
        word = reverse_word_index[word_num]
        embeddings = weights[word_num]
        out_m.write(word + "\n")
        out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [22]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
